<a href="https://colab.research.google.com/github/bhoomipuri/bhoomipuri/blob/main/Fake%20News%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install seaborn scikit-learn

In [11]:
news_path = '/content/drive/MyDrive/Fake-News-Detection-main/news.csv'

In [12]:
df = pd.read_csv(news_path)

In [13]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [14]:
df.shape

(6335, 4)

In [15]:
df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [16]:
labels = df.label

In [17]:
labels.head()

,label
0,FAKE
1,FAKE
2,REAL
3,FAKE
4,REAL


In [18]:
from sklearn.model_selection import train_test_split

In [20]:
x_train , x_test , y_train , y_test = train_test_split(df["text"] , labels , test_size = 0.2 , random_state = 20 )

In [21]:
x_train.head()

,text
4741,"NAIROBI, Kenya — President Obama spoke out Sun..."
2089,"Killing Obama administration rules, dismantlin..."
4074,"Dean Obeidallah, a former attorney, is the hos..."
5376,WashingtonsBlog \nCNN’s Jake Tapper hit the ...
6028,Some of the biggest issues facing America this...


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

In [23]:
vector = TfidfVectorizer(stop_words = 'english' , max_df = 0.7 )

In [24]:
tf_train = vector.fit_transform(x_train)
tf_test = vector.transform(x_test)

In [25]:
pac = PassiveAggressiveClassifier( max_iter=50 )
pac.fit( tf_train , y_train )

PassiveAggressiveClassifier(max_iter=50)

In [26]:
from sklearn.metrics import accuracy_score , confusion_matrix
y_pred = pac.predict(tf_test)

In [27]:
score = accuracy_score(y_test , y_pred )

In [28]:
print(f"accuracy : {round(score*100,2)}%")

accuracy : 94.4%


In [29]:
confusion_matrix(y_test , y_pred , labels = [ 'FAKE' , 'REAL' ] )

array([[622,  26],
       [ 45, 574]])

In [30]:
import pickle
filename = 'finelized_model.pkl'
pickle.dump(pac,open(filename, 'wb'))



In [31]:
filename = 'vectorizer.pkl'
pickle.dump(vector , open(filename , 'wb'))

In [ ]:

from flask import Flask, request, render_template
import pickle


vector = pickle.load(open("/content/vectorizer.pkl", 'rb'))
model = pickle.load(open("/content/finelized_model.pkl", 'rb'))

app = Flask(__name__)

@app.route('/')
def home():
    return render_template("/content/index.html")

@app.route('/prediction', methods=['GET', 'POST'])
def prediction():
    if request.method == "POST":
        news = str(request.form['news'])

        predict = model.predict(vector.transform([news]))[0]
        print(predict)

        return render_template("/content/prediction.html", prediction_text="News headline is -> {}".format(predict))


    else:
        return render_template("/content/prediction.html")


if __name__ == '__main__':
    app.debug = True
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
